In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

#Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# # dx = 1 km; Np = 1M; Nt = 5 min
# data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc') #***
# parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc') #***
# res='1km';t_res='5min';Np_str='1e6'

# dx = 1km; Np = 50M
#Importing Model Data
check=False
dir2='/home/air673/koa_scratch/'
data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 100M
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_100M.nc') #***
# res='1km'; t_res='1min'; Np_str='100e6'


# dx = 250 m
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_250m.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_250m.nc') #***

In [ ]:
#LOADING IN H5 VARIABLES
file_path = dir + 'Variable_Calculation/' + f'theta_e_{res}_{t_res}.h5'
th_e_ds = xr.open_dataset(file_path, phony_dims='sort',engine='h5netcdf')  # or engine='netcdf4'
th_e_ds = th_e_ds.rename({
    'phony_dim_0': 'time',
    'phony_dim_1': 'zh',
    'phony_dim_2': 'yh',
    'phony_dim_3': 'xh'
})

file_path = dir + 'Variable_Calculation/' + '2D_Moisture_Convergence' + f'_{res}_{t_res}' + '.h5'
hmc_ds = xr.open_dataset(file_path, phony_dims='sort',engine='h5netcdf')  # or engine='netcdf4'
hmc_ds = hmc_ds.rename({
    'phony_dim_0': 'time',
    'phony_dim_1': 'zh',
    'phony_dim_2': 'yh',
    'phony_dim_3': 'xh'
})

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
#DOMAIN SUBSETTING
############################################################
ocean_percent=2/8

left_to_coast=data['xh'][0]+(data['xh'][-1]-data['xh'][0])*ocean_percent
where_coast_xh=np.where(data['xh']>=left_to_coast)[0][0]#-25
where_coast_xf=np.where(data['xf']>=left_to_coast)[0][0]#-25
end_xh=len(data['xh'])-1-50
end_xf=len(data['xf'])-1-50

print(f'x in {0}:{where_coast_xh-1} FOR SEA')
print(f'x in {where_coast_xh}:{end_xh} FOR LAND')
# t_end=78 
# if res=='250m':t_end=410
# print(f't in {0}:{t_end} (6.5 hours)')
if t_res=="5min":
    t_start=36 # 9:00 am (3 hours in)
if t_res=="1min":
    t_start=36*5 # 9:00 am (3 hours in)
print(f't in {t_start}:end (8 hours)')


#SUBSETTING CODE
data=data.isel(time=slice(t_start,None),xh=slice(where_coast_xh,end_xh+1),xf=slice(where_coast_xf,end_xf+1))
th_e_ds=th_e_ds.isel(time=slice(t_start,None),xh=slice(where_coast_xh,end_xh+1))
hmc_ds=hmc_ds.isel(time=slice(t_start,None),xh=slice(where_coast_xh,end_xh+1))

In [ ]:
#JOB ARRAY SETUP
job_array=False;index_adjust=0
job_array=True

if job_array==True:

    num_jobs=180 #how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100 #***
    total_elements=len(data['time']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=3
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    print(f'start_job = {start_job}, end_job = {end_job}')

In [ ]:
if job_array==True:
    #Indexing Array with JobArray
    data=data.isel(time=slice(start_job,end_job))
    parcel=parcel.isel(time=slice(start_job,end_job))
    th_e_ds=th_e_ds.isel(time=slice(start_job,end_job))
    hmc_ds=hmc_ds.isel(time=slice(start_job,end_job))

In [ ]:
def call_variables(): #***
    variable='winterp'; w_data=data[variable].data
    variable='qv'; qv_data=data[variable].data # get qc data
    variable='qc'; qc_data=data[variable].data # get qc data
    variable='qi'; qi_data=data[variable].data # get qc data
    qc_plus_qi=qc_data+qi_data
    variable='th'; th_data=data[variable].data # get qc data
    variable='buoyancy'; buoyancy_data=data[variable].data # get qc data

    variable='theta_e'; theta_e_data=th_e_ds[variable].data
    variable='conv'; HMC_data=hmc_ds[variable].data
        
    return w_data,qv_data,qc_data,qi_data,qc_plus_qi,th_data,buoyancy_data,theta_e_data,HMC_data

print('calling variables')
[w_data,qv_data,qc_data,qi_data,qc_plus_qi,th_data,buoyancy_data,theta_e_data,HMC_data]=call_variables()
print('done')

In [ ]:
check_memory(globals())

In [ ]:
##########################################################################################
#Functions
# Full Profile function makes profile together for all timesteps. AveragedProfiles funciton takes the final mean of the combined profile.

In [ ]:
# Thresholds
w_thresh1 = 0.1
w_thresh2 = 0.5
qcqi_thresh = 1e-6

def final_profile(vars_list, type):
    zhs = data['zh'].values
    profiles = {}  # Store profiles for all variables

    # Initialize profiles for each variable
    for var in vars_list:
        profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
        profiles[var][:, 2] = zhs

    ###DOMAIN_SUBSETTING
    if end_job < t_start:
        return profiles
    ####################

    # Threshold mask
    if type == "general":
        where_updraft = (w_data >= w_thresh1) & (qc_plus_qi < qcqi_thresh)
    elif type == "cloudy":
        where_updraft = (w_data >= w_thresh2) & (qc_plus_qi >= qcqi_thresh)
    t_ind, z_ind, y_ind, x_ind = np.where(where_updraft)

    # Variable selection dictionary
    var_data = {
        'w': w_data, 'qv': qv_data, 'qc': qc_plus_qi, 'qi': qi_data,
        'th': th_data, 'th_e': theta_e_data, 'buoyancy': buoyancy_data, 'HMC': HMC_data
    }

    # Iterate over each variable in vars_list and bin the data
    for var in vars_list:
        masked_data = var_data[var][where_updraft]
        np.add.at(profiles[var][:, 0], z_ind, masked_data)
        np.add.at(profiles[var][:, 1], z_ind, 1)

    return profiles

In [ ]:
##########################################################################################
#Running

In [ ]:
dir2=dir+'Project_Algorithms/Domain_Profiles/'

for type in ['general','cloudy']:
    print(f"working on {type} type\n")
    
    vars_list = ['w', 'qv', 'qc', 'qi', 'th', 'th_e', 'buoyancy', 'HMC']
    profiles = final_profile(vars_list, type)
    
    #Saving eulerian_profiles
    import h5py
    
    if type == "general":
        output_file = dir2+f'job_out/general_eulerian_profiles_{res}_{t_res}_{Np_str}'
    elif type == "cloudy":
        output_file = dir2+f'job_out/cloudy_eulerian_profiles_{res}_{t_res}_{Np_str}'

    if job_array==True:
        output_file+=f'_{job_id}.h5'
    elif job_array==False:
        output_file+=f'.h5'
    
    with h5py.File(output_file, 'w') as f:
        for var in profiles:
            profile_var = profiles[var]
            f.create_dataset(f'profile_{var}', data=profile_var, compression="gzip")

In [ ]:
check_memory(globals())

In [ ]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE WHEN JOB ARRAY IS RUNNING
# recombine=True

In [ ]:
if recombine==True:
    dir2=dir+'Project_Algorithms/Domain_Profiles/'
    
    types=['general','cloudy']
    for type in types:
        #MAKING OUTPUT FILE PATH
        if type == "general":
            output_file = dir2+f'job_out/general_eulerian_profiles_{res}_{t_res}_{Np_str}.h5' 
        elif type == "cloudy":
            output_file = dir2+f'job_out/cloudy_eulerian_profiles_{res}_{t_res}_{Np_str}.h5'
        
        #MAKING PROFILES DICTIONARY
        zhs = data['zh'].values
        profiles = {}  #  Store profiles for all variables
        vars_list = ['w', 'qv', 'qc', 'qi', 'th', 'th_e', 'buoyancy', 'HMC']
        for var in vars_list:
            profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
            profiles[var][:, 2] = zhs 
        
        num_jobs=180
        for job_id in np.arange(1,num_jobs+1):
            if np.mod(job_id,10)==0: print(f"job_id = {job_id}")
            #CALLING IN DATA
            if type == "general":
                input_file = dir2+f'job_out/general_eulerian_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5' 
            elif type == "cloudy":
                input_file = dir2+f'job_out/cloudy_eulerian_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5'
    
            #COMPILING PROFILES
            with h5py.File(input_file, 'r') as f:
                for var in vars_list:  
                    profiles[var][:,0:1+1]+=f[f'profile_{var}'][:,0:1+1]
        
        #SAVING INTO FINAL FORM
        with h5py.File(output_file, 'w') as f:
            for var in profiles:
                profile_var = profiles[var]
                f.create_dataset(f'profile_{var}', data=profile_var, compression="gzip")